In [ ]:
import sys
import colorsys

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

sys.path.append('..')

from src.visualization.visualize import make_df_colormap
from src.data.import_data_pks import hierarchize_data

In [ ]:
data = pd.read_parquet("../data/processed/pks.parquet")    
colormap = {k: grp.color.iloc[0] for k, grp in data.groupby("key")}

In [ ]:
selected_keys_states = [{"state": "Bund", "key": i} for i in ["300000"]]

df_ts = pd.concat([
        data.loc[(data.state.eq(i["state"])) & (data.key.eq(i["key"]))] for i in selected_keys_states
    ])

df_ts = df_ts[["key", "state", "year",
                   "label", "count", "clearance", "color"]]
df_ts["unsolved"] = df_ts["count"] - df_ts["clearance"]
df_ts["clearance_rate"] = df_ts.apply(lambda r: r["clearance"] / r["count"] * 100, axis=1)

In [ ]:
df_ts

In [ ]:
data = data.loc[(data.year.eq(2021) & (data.state.eq("Bund")))]
data = hierarchize_data(data)

fig = px.sunburst(data,
                  names='key',
                #   values='one',
                  parents='parent',
                  color="key",
                  color_discrete_map=colormap,
                  hover_data=['label', 'key', 'count'],
                  maxdepth=2,
                  height=750,
                  width=750)

fig.update_layout(margin=dict(t=15, r=15, b=15, l=15))
fig.update_coloraxes(showscale=False)

In [ ]:
data = pd.read_parquet("../data/processed/pks.parquet")
all_years = data.year.unique()



In [ ]:
a = "3***00"  # soll kleiner sein
b = "310*00"


def star_lt(x: str, y: str) -> bool:
    """
    Helper: compare keys even when they contain asterisks.
    * couns as "less than zero".
    """
    # einfach:
    try:
        if int(x) < int(y):
            return True

    # Handarbeit:
    except ValueError:
        testing_range = min(len(x), len(y))

        for i in range(testing_range):
            if x[i] == y[i]:
                continue

            if x[i] in "0123456789" and y[i] == "*":
                return False
            
            if x[i] == "*" and y[i] in "0123456789":
                return True

        return False

In [ ]:
star_lt(b, a)